In [1]:
import xarray as xr
import pandas as pd
import panel as pn
import numpy as np
import panel as pn
from holoviews import opts
import holoviews as hv
from thredds_crawler.crawl import Crawl
import datetime as dt
from functools import partial
from get_data import get_urls_mod, get_urls_obs, get_model_data, get_obs_data
from scipy import stats

In [2]:
pn.extension()
hv.extension("bokeh")

# Use-case: Scatter 1

## Introduction 

This notebook contains the code for generating a scatter plot of model vs. observational data for a chosen site, model, variable and time.

The code for fetching the data can be found in get_data.py 

This plotting tool as been generated for the Year Of Polar Prediction site Model Intercomparison Project, YOPPsiteMIP

## Code 

The following cell contains the function responsible for the merging of the dataframes containing the model and observation data. This gets rid of any points between them where there is no direct timestep match. 

In [3]:
def merge_data(var_obs,var_mod,variable):
    #rename column names to easier differentiate between values when combined
    var_obs = var_obs.rename(columns={variable:"var_obs"})
    var_mod = var_mod.rename(columns={variable:"var_mod"})
    
    #merges the dataframes based on matching time points, gets rid of all mismatches for time
    if len(var_obs.index) < len(var_mod):
        merged = var_obs.merge(var_mod, left_index=True, right_index=True, how='inner')
    else:
        merged = var_mod.merge(var_obs, left_index=True, right_index=True, how='inner')

    return merged

The following cell contains the code for the widgets used for user selection
TO DO: widget options should be updated dynamically

In [4]:
#selection of site
site_name = pn.widgets.Select(
    name="Site", options=['sodankyla','tiksi'],value = 'tiksi', margin=(0, 20, 0, 0)
)

#selection of model
model_name = pn.widgets.Select(
    name="model", options=['AROME-Arctic','ifs-ecmwf'],value='ifs-ecmwf', margin=(0, 20, 0, 0)
)

#user selection variable
variable = pn.widgets.Select(
    name="Variables", options=['tas','snd'], margin=(0, 20, 0, 0)
)

#user selection start and end date only sop 1 as of right now
dates = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=dt.datetime(2018, 2, 1), end=dt.datetime(2018, 3, 31),
    value=(dt.datetime(2018, 2, 1), dt.datetime(2018, 2, 2))
)

#user selection start time for model 
start_time = pn.widgets.Select(
    name="Start time", options=['00','12'], margin=(0, 20, 0, 0)
)

#user selection of which day should be fetched from the model files
#for concat
concat_day = pn.widgets.Select(
    name="Model day selected", options={1:0,2:1,3:2}, margin=(0,20,0,0)
)

#TO DO: add dependecy so that 1-1 toggles line reg off and vice versa
line_reg = pn.widgets.Checkbox(name='Linear regression')
one_one = pn.widgets.Checkbox(name='1 to 1 line')

The following cell contains the function responsible for creating a scatter plot if the model data vs. observations. It also, if selected, adds a 1 to 1 line or linear regression. 
TO DO: add legend

In [5]:
#plots the data as a scatter plot with a linear regression 
#axies are kept constant 
def plot_scatter(data,var,line_reg,one_one,model,site):
    
    scatter = hv.Points(data)
    
    if line_reg:
        slope, intercept, r, p, se = stats.linregress(data.var_obs,data.var_mod)
        linereg = hv.Slope(slope, intercept)
        linereg.opts(color = 'red')
        rsquare = hv.Text(256, 246,"R²: " +str(np.round(r**2,4)))
        scatter_1 = scatter * linereg * rsquare
        
    elif one_one:
        line = hv.Slope(1,0)
        line.opts(color='grey')
        
        scatter_1 = scatter*line
        
    else:
        scatter_1 = scatter
    
    bounds_upper = max(data.max().var_obs,data.max().var_mod)
    bounds_lower = min(data.min().var_obs,data.min().var_mod)
    
    bounds = (bounds_lower,bounds_upper)
     
    scatter_1.opts(title="Scatter 1: "+var+' '+model+' '+site,
        ylabel="model",
        xlabel="observation",
        xlim = bounds,
        ylim = bounds,
        width=400,
        height=400
    )
    
    return scatter_1

The following cell contains the main function, updated by the user input from widgets, and calling the relevant functions to produce and return a scatter plot of the data based on user selections.

In [6]:
@pn.depends(variable,dates,start_time,concat_day,line_reg,one_one,model_name,site_name)
def scatter_plot(variable,dates,start_time,concat_day,line_reg,one_one,model_name,site_name):
    urls_mod = get_urls_mod(dates[0],dates[1],start_time,model_name,site_name,[variable],concat_day)
    url_obs = get_urls_obs(site_name, "timeSeries","1")
    out_type='concatenated'
    var_mod = get_model_data(out_type,urls_mod,concat_day).to_dataframe()
    var_obs = get_obs_data(url_obs,dates[0],dates[1],[variable]).to_dataframe()
    
    merged = merge_data(var_obs,var_mod,variable)
    
    plot = plot_scatter(merged,variable,line_reg,one_one,model_name,site_name)
    return plot

The following cell contains the function responsible for deploying the application

In [8]:
pn.Column("scatter 1 sop 1", scatter_plot,pn.Row(pn.Column(model_name,site_name,variable,start_time),pn.Column(dates,concat_day,one_one,line_reg)),width_policy="max").servable()


Unexpected exception formatting exception. Falling back to standard exception


Note:Caching=1
Error:curl error: Problem with the SSL CA cert (path? access rights?)
curl error details: 
Traceback (most recent call last):
  File "/home/jo/.local/lib/python3.10/site-packages/xarray/backends/file_manager.py", line 201, in _acquire_with_cache_info
    file = self._cache[self._key]
  File "/home/jo/.local/lib/python3.10/site-packages/xarray/backends/lru_cache.py", line 55, in __getitem__
    value = self._cache[key]
KeyError: [<class 'netCDF4._netCDF4.Dataset'>, ('https://thredds.met.no/thredds/dodsC/alertness/YOPP_supersite/ifs-ecmwf/tiksi/tiksi_ifs-ecmwf_2018020100.nc?time,tas',), 'r', (('clobber', True), ('diskless', False), ('format', 'NETCDF4'), ('persist', False))]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jo/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_23627/2985